In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
DIR = '/content/drive/MyDrive/ToiUuLapLich/data'
%cd '/content/drive/MyDrive/ToiUuLapLich/data'

/content/drive/MyDrive/ToiUuLapLich/data


## Data Preprocessing

In [3]:
def read_data(data_file_name):
  with open(data_file_name, "r") as f:
    r = f.readlines()
    number_subject = int(r[0])
    student_per_subject = [int(u) for u in r[1][:-1].split(" ")]
    number_room = int(r[2])
    place_per_room = [int(u) for u in r[3][:-1].split(" ")]
    number_constrain = int(r[4])

    student_per_subject.sort(reverse=True)
    student_per_subject = np.asarray(student_per_subject)
    place_per_room.sort(reverse=True)
    place_per_room = np.asarray(place_per_room)

    if(r[-1][-1] != '\n'):
      r[-1] += "\n"

    constrain = []
    for i in range(number_constrain):
      each_constrain = [int(u) for u in r[5+i][:-1].split(" ")]
      constrain.append(each_constrain)

    matrix_constrain = np.zeros((number_subject, number_subject))
    for iter in constrain:
      i = iter[0]
      j = iter[1]
      matrix_constrain[i][j] = 1
      matrix_constrain[j][i] = 1
        
    return number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain

# TEST
number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(f'{DIR}/data_10_2_(0).txt')


# Greedy

In [4]:
def greedy(data_path):
  # Read data 
  number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(data_path)

  # Solve
  solve = []
  kip = 0
  subject_checked = np.zeros((number_subject,))

  while(np.sum(subject_checked) < number_subject):
    kip += 1
    this_kip = []
    iter_room = 0
    iter_sub = 0
    conflict = np.empty((0,), dtype=int)
    while((iter_sub < len(student_per_subject)) and (iter_room < len(place_per_room))):
      if (subject_checked[iter_sub] == 1) or (iter_sub in conflict):  # already checked
        iter_sub += 1;
        continue;
      else: # unchecked
        if(place_per_room[iter_room] >= student_per_subject[iter_sub]): # fit the room -> checked
          subject_checked[iter_sub] = 1
          # TO-DO ADD CONFLICT
          new_conflict = np.where(matrix_constrain[iter_sub] == 1)[0]
          conflict = np.append(conflict, new_conflict)
          conflict = np.unique(conflict)
          # ADD CONFLICT
          this_kip.append([iter_sub, iter_room])
          iter_room += 1
          iter_sub += 1
          
        else:
          iter_sub += 1
    solve.append(this_kip)

  return kip, solve


In [9]:
def check_order(data_path, order):
  number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(data_path)
  iter_order = 0
  result = []
  while(iter_order < len(order)):
    this_kip = []
    iter_room = 0
    conflict = np.empty((0,), dtype=int)

    while((iter_room < number_room) and (iter_order < number_subject)):
      if((place_per_room[iter_room] < student_per_subject[order[iter_order]]) or (order[iter_order] in conflict)):
        break
      else:
        # TO-DO ADD CONFLICT
        new_conflict = np.where(matrix_constrain[order[iter_order]] == 1)[0]
        conflict = np.append(conflict, new_conflict)
        conflict = np.unique(conflict)
        # ADD CONFLICT
        this_kip.append([order[iter_order], iter_room])
        iter_room += 1
        iter_order += 1
    result.append(this_kip)

  return len(result), result


In [10]:
def check_order_read(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain, order):
  iter_order = 0
  result = []
  while(iter_order < len(order)):
    this_kip = []
    iter_room = 0
    conflict = np.empty((0,), dtype=int)

    while((iter_room < number_room) and (iter_order < number_subject)):
      if((place_per_room[iter_room] < student_per_subject[order[iter_order]]) or (order[iter_order] in conflict)):
        break
      else:
        # TO-DO ADD CONFLICT
        new_conflict = np.where(matrix_constrain[order[iter_order]] == 1)[0]
        conflict = np.append(conflict, new_conflict)
        conflict = np.unique(conflict)
        # ADD CONFLICT
        this_kip.append([order[iter_order], iter_room])
        iter_room += 1
        iter_order += 1
    result.append(this_kip)

  return len(result), result


# Solve


In [11]:
def solve_prob(data_file_name):
  DIR = '/content/drive/MyDrive/ToiUuLapLich/data'
  data_path = os.path.join(DIR, data_file_name)
  return greedy(data_path)

In [12]:
kip, solution = solve_prob("data_10_2_(1).txt")

In [13]:
print(kip)
assert (kip == len(solution))
print(solution)

6
[[[0, 0], [5, 1]], [[1, 0], [6, 1]], [[2, 0], [7, 1]], [[3, 0], [8, 1]], [[4, 0]], [[9, 0]]]


In [14]:
order = [0,5,1,6,2,7,3,8,4,9]
kip, solve = check_order("data_10_2_(1).txt", order)
print(kip)
print(solve)

6
[[[0, 0], [5, 1]], [[1, 0], [6, 1]], [[2, 0], [7, 1]], [[3, 0], [8, 1]], [[4, 0]], [[9, 0]]]


# Local Search

In [17]:
def get_order(solve):
  order = []
  for kip in solve:
    for i in kip:
      order.append(i[0])
  return order

In [35]:
# init greedy solution
data_file = "data_10_2_(1).txt"
kip, solve = solve_prob(data_file)
order = get_order(solve)
order

[0, 5, 1, 6, 2, 7, 3, 8, 4, 9]

In [18]:
def swap(order, r1, r2):
  temp = order[r1]
  order[r1] = order[r2]
  order[r2] = temp
  return 

In [22]:
order = np.arange(number_subject)
order

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [45]:
print(order)
swap(order, 0, 1)
print(order)

[5, 0, 1, 6, 2, 7, 3, 8, 4, 9]
[0, 5, 1, 6, 2, 7, 3, 8, 4, 9]


## Naive 

In [23]:
import random

def naive_local_search(data_file, number_loop):
  number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(data_file)

  # init greedy solution
  # len_kip, solve = solve_prob(data_file)
  # order = get_order(solve)
  # result_len = len_kip

  # init random solution 
  order = np.arange(number_subject)
  len_this_sol, this_sol = check_order_read(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain, order)
  result_len = len_this_sol


  # loop local search
  for i in range(number_loop):
    r1 = random.randint(0, number_subject-1)
    r2 = random.randint(0, number_subject-1)
    if (r1 == r2): 
      continue
    else:
      swap(order, r1, r2)
      len_this_sol, this_sol = check_order_read(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain, order)
      if(len_this_sol < result_len):
        result_len = len_this_sol
      else:
        swap(order, r1, r2)

  return result_len, order


In [28]:
data_file = "data_10_2_(1).txt"
result_len, order = naive_local_search(data_file, number_loop = 40)
print(result_len)
print(order)

6
[0 4 2 6 1 5 3 7 8 9]


In [53]:
len, solve = check_order(data_file, order)
print(len)
print(solve)

5
[[[0, 0], [5, 1]], [[4, 0], [6, 1]], [[2, 0], [7, 1]], [[3, 0], [8, 1]], [[1, 0], [9, 1]]]
